In [1]:
import os
from langchain_chroma import Chroma

In [2]:
import sys
print(sys.executable)

/Users/zsk4gm/Desktop/resilience_education/env/bin/python3


## Initiate Chroma DB with saved embeddings

In [3]:
# Create the file path to the 'embeddings' folder
current_directory = os.getcwd()
persist_directory = os.path.join(current_directory, 'embeddings')

In [4]:
# Load Embedding Model 
from langchain_community.embeddings import HuggingFaceEmbeddings

model_name = "multi-qa-mpnet-base-dot-v1"
model_kwargs = {'device': 'cpu'}
encode_kwargs = {'normalize_embeddings': False}
hf = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs=model_kwargs,
    encode_kwargs=encode_kwargs
)

/Users/zsk4gm/Desktop/resilience_education/env/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/Users/zsk4gm/Desktop/resilience_education/env/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [5]:
# Create the vector store and specify the persist directory\n",
vectorstore = Chroma(persist_directory=persist_directory, embedding_function=hf)

## Load LLM

In [6]:
from langchain_community.llms import Ollama
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

class CustomOllama(Ollama):
    def __init__(self, model, callback_manager=None, stop=None, temperature=0.2):
        super().__init__(model=model, stop=stop, temperature=temperature)
        self.callback_manager = callback_manager

# Initialize the CallbackManager
callback_manager = CallbackManager([StreamingStdOutCallbackHandler()])

# Create an instance of the CustomOllama class
llm = CustomOllama(
    model="llama3",
    callback_manager=callback_manager
)

In [98]:
#test 
prompt = ["Why is the sky blue?"]  # Prompt should be a list of strings

# Generate text using the Ollama model
generated_text = llm.generate(prompts=prompt)


In [8]:
response = generated_text.generations[0][0].text
print(response)

The sky appears blue because of a phenomenon called Rayleigh scattering, named after the British physicist Lord Rayleigh. Here's what happens:

1. **Sunlight**: When sunlight enters Earth's atmosphere, it contains all the colors of the visible spectrum (red, orange, yellow, green, blue, indigo, and violet).
2. **Molecules**: The atmosphere is made up of tiny molecules of gases like nitrogen (N2) and oxygen (O2). These molecules are much smaller than the wavelength of light.
3. **Scattering**: When sunlight hits these molecules, it scatters in all directions. This scattering effect is more pronounced for shorter wavelengths (like blue and violet) than longer wavelengths (like red and orange).
4. **Blue dominance**: As a result of this scattering, the shorter wavelengths of light (blue and violet) are distributed throughout the atmosphere, reaching our eyes from all directions.
5. **Our perception**: Our brains perceive this scattered blue light as the color of the sky.

In summary, the 

## Setup RAG

### Instantiate retriever

In [55]:
plain_retriever = vectorstore.as_retriever()

### Chain with chat history 

1.) Step 1: define a subchain that contextualizes the user's query by modifying the system prompt to reformulate the question using the chat history 
    a.) In the subchain we'll use a messagesplaceholder to insert a list of messages as the chat history 
    b.) use helper function create_history_aware_retriever that reverts to regular retrieval chain if no chat history.
2.) 

In [58]:
# define a subchain that takes histroical messages and latest user question 
from langchain.chains import create_history_aware_retriever 
from langchain_core.prompts import MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser

contextualize_q_system_prompt = '''Given a chat history and the latest user question 
which might reference context in the chat history, formulate a standalone question which can 
be understood without a chat history. Do NOT answer the question, just reformulate it if needed 
and otherwise return it as is.'''

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ('system', contextualize_q_system_prompt), 
        MessagesPlaceholder('chat_history'),
        ('human', '{input}'), 
    ] 
) 

contextualize_q_chain = contextualize_q_prompt | llm | StrOutputParser()

In [89]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
'''You are a compassionate legal expert tasked with translating Virginia legal restrictions into helpful plaintext for jobseekers with felony or misdemeanor convictions. Your goal is to help them understand what jobs or certifications they can pursue while maintaining a supportive and encouraging tone.

1. Begin with a brief disclaimer: Remind the user that you cannot provide personalized legal advice and that all information is general. Emphasize the importance of consulting with a legal professional for specific guidance.

2. If not provided, ask for relevant details about the user's specific situation (e.g., type of conviction, how long ago it occurred) to provide more accurate information.

3. Use only the following sections of the Virginia legal code to answer the user's query:
   {context}

4. Provide a clear and concise answer addressing the user's query, including as many relevant details as possible from the context. Always cite the specific section of the code you're referencing.

5. If there are restrictions that employers can waive, describe those options clearly.

6. If you are uncertain about any of the restrictions or if none of the sections of the code answer the query, state your uncertainty and recommend consulting a legal professional.

7. Suggest similar jobs or certifications that the user can legally pursue with their conviction in Virginia. Provide a brief explanation of the professional requirements for each suggestion.

8. Encourage the user to conduct further research and provide suggestions for additional resources they can consult (e.g., state employment agencies, legal aid organizations).

9. Conclude with a supportive message, reminding the user that there are often pathways to employment despite past convictions.

10. For follow-up questions:
    a. Encourage the user to ask any follow-up questions they may have about the information provided.
    b. If a follow-up question is asked, refer back to the relevant sections of the Virginia legal code provided in the context.
    c. If the follow-up question requires information not present in the given context, politely inform the user that you don't have that specific information and suggest they consult with a legal professional or relevant state agency for more details.
    d. Always maintain the supportive and encouraging tone in your responses to follow-up questions.

Remember to maintain a balance between providing accurate information and offering encouragement to the jobseeker throughout the initial response and any follow-up interactions.
''')

prompt_plain = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder('chat_history'),
        ("human", "{input}"),
    ]
)
prompt_plain.pretty_print()

================================ System Message ================================

You are a compassionate legal expert tasked with translating Virginia legal restrictions into helpful plaintext for jobseekers with felony or misdemeanor convictions. Your goal is to help them understand what jobs or certifications they can pursue while maintaining a supportive and encouraging tone.

1. Begin with a brief disclaimer: Remind the user that you cannot provide personalized legal advice and that all information is general. Emphasize the importance of consulting with a legal professional for specific guidance.

2. If not provided, ask for relevant details about the user's specific situation (e.g., type of conviction, how long ago it occurred) to provide more accurate information.

3. Use only the following sections of the Virginia legal code to answer the user's query:
   {context}

4. Provide a clear and concise answer addressing the user's query, including as many relevant details as possible

In [155]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from typing import List, Dict
from langchain_core.documents import Document
from langchain.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.callbacks import StdOutCallbackHandler
from langchain_community.chat_message_histories import SQLChatMessageHistory

custom_document_prompt = PromptTemplate.from_template(
    '''
Content: {page_content}

Metadata:
- Chapter Number: {ChapterNum}
- Chapter Name: {ChapterName}
- Article Number: {ArticleNum}
- Article Name: {ArticleName}
- Section Number: {SectionNumber}
- Section Title: {SectionTitle}
'''
)
        
question_answer_chain = create_stuff_documents_chain(llm=llm, prompt=prompt_plain, document_prompt=custom_document_prompt, output_parser=StrOutputParser())

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection_string="sqlite:///sqlite.db"
    ),
    input_messages_key="input",
    history_messages_key="chat_history",
    output_messages_key="answer"
)

### Test with prompt

In [154]:
# This is where we configure the session id
config = {"configurable": {"session_id": "<SQL_SESSION_ID>"}}

In [156]:
# Assuming you have a string for your prompt template
query = "Can I be a teacher with a violent crime on my record?"

In [157]:
import time 
start_time = time.time()

result = conversational_rag_chain.invoke(
    {"input": query},
    config=config
)

# Stop the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

In [158]:
print(elapsed_time) 

32.10410284996033


In [159]:
print(result['answer'])

I'm happy to help you understand Virginia's laws regarding teaching with a felony conviction. Please note that I am not a legal expert, and this information is general in nature. For specific guidance, it's essential to consult with a legal professional.

To provide more accurate information, could you please share the details of your violent crime conviction? This will help me better understand the context and provide a more tailored response.

Based on the provided Virginia code sections (§ 22.1-296.1 and § 22.1-307), it appears that having a violent felony conviction may disqualify you from becoming a teacher in Virginia. According to Section B, no school board shall employ any individual who has been convicted of any violent felony set forth in the definition of barrier crime (§ 19.2-392.02) or any offense involving the sexual molestation, physical or sexual abuse, or rape of a child.

Additionally, Section C states that even if you've had your civil rights restored by the Governor

In [160]:
print(result['context'])

[Document(metadata={'ArticleName': 'Terms of Employment Generally', 'ArticleNum': '2', 'ChapterName': 'Teachers, Officers and Employees', 'ChapterNum': '15', 'Hrefs': '/vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; /vacode/19.2-392.02/; http://lis.virginia.gov/cgi-bin/legp604.exe?961+ful+CHAP0960; http://lis.virginia.gov/cgi-bin/legp604.exe?971+ful+CHAP0103; http://lis.virginia.gov/cgi-bin/legp604.exe?031+ful+CHAP0723; http://lis.virginia.gov/cgi-bin/legp604.exe?061+ful+CHAP0790; http://lis.virginia.gov/cgi-bin/legp604.exe?071+ful+CHAP0245; http://lis.virginia.gov/cgi-bin/legp604.exe?071+ful+CHAP0431; http://lis.virginia.gov/cgi-bin/legp604.exe?081+ful+CHAP0555; http://lis.virginia.gov/cgi-bin/legp604.exe?201+ful+CHAP0877; http://lis.virginia.gov/cgi-bin/legp604.exe?231+ful+CHAP0670; http://lis.virginia.gov/cgi-bin/legp604.exe?231+ful+CHAP0703', 'SectionNumber': '22.1-296.1', 'SectionTitle': 'Data on convictions for certain

## Test Remembering

In [161]:
result_remember = conversational_rag_chain.invoke({"input": 'What is in our chat history?'}, 
                                        config=config)

In [162]:
print(result_remember['answer'])

Our chat history includes the following:

1. Your initial question about whether you can be a teacher with a violent crime on your record.
2. My response, which provided general information about Virginia's laws regarding teaching with a felony conviction and mentioned that I am not a legal expert, but rather a compassionate legal expert tasked with translating Virginia legal restrictions into helpful plaintext.

I also asked for more details about your violent crime conviction to better understand the context and provide a more tailored response.


In [165]:
query2 = "What jobs can I get in healthcare if I've been convicted of a drug offense?"
result2 = conversational_rag_chain.invoke({"input": query2}, 
                                         config=config)

In [166]:
print(result2['answer'])

Based on the Virginia Code (§ 54.1-3408), having a drug-related conviction may disqualify you from working in certain healthcare roles. However, there are still many opportunities available that don't require a waiver.

Here are some potential jobs in healthcare that you might be qualified for:

1. Medical Records Technician: You can work in medical offices, hospitals, or clinics, managing and maintaining patient records.
2. Health Educator: With your experience, you could educate patients about healthy habits, disease prevention, and management of chronic conditions.
3. Medical Assistant: As a medical assistant, you'll perform administrative tasks, take vital signs, and assist healthcare professionals with patient care.
4. Pharmacy Technician: You can work in pharmacies, hospitals, or clinics, assisting pharmacists with dispensing medications and providing customer service.
5. Health Information Manager: In this role, you'll oversee the collection, analysis, and reporting of health da

In [163]:
query2a = "What jobs can I get in healthcare if I've been convicted of a drug offense and don't require a waiver?"
result2a = conversational_rag_chain.invoke({"input": query2a}, 
                             config=config)
print(result2a['answer'])

I'm happy to help you explore healthcare job opportunities in Virginia despite your drug-related conviction. Please note that I am not a legal expert, and this information is general in nature. For specific guidance, it's essential to consult with a legal professional.

According to the Virginia Code (§ 54.1-3408), having a drug-related conviction may disqualify you from working in certain healthcare roles. However, there are still many opportunities available that don't require a waiver.

Here are some potential jobs in healthcare that you might be qualified for:

1. Medical Records Technician: You can work in medical offices, hospitals, or clinics, managing and maintaining patient records.
2. Health Educator: With your experience, you could educate patients about healthy habits, disease prevention, and management of chronic conditions.
3. Medical Assistant: As a medical assistant, you'll perform administrative tasks, take vital signs, and assist healthcare professionals with patient 

In [ ]:
query2b = "What certifications or trainings could I get"
result2b = conversational_rag_chain.invoke({"input": query2b},
                                          config=config)
print(result2b['answer'])

In [21]:
query3 = "What is Taylor Swift's latest album called?"
result3 = conversational_rag_chain.invoke({"input": query3}, 
                                         config=config)
print(result3['answer'])

I'm happy to help you with your query! However, since this is a legal question related to Virginia laws, I'll provide an answer based on the provided metadata.

Unfortunately, there doesn't seem to be any information in the provided code that directly addresses job opportunities or restrictions for individuals with convictions. The section "Makehamized mortality table" (55.1-504) appears to be related to insurance and life expectancy calculations, rather than employment or criminal records.

If you're looking for guidance on employment opportunities despite a conviction, I recommend consulting the Virginia Department of Corrections' Reentry Program or the Virginia Employment Commission's resources for individuals with criminal convictions. These organizations may provide valuable information on job training programs, education, and employment services that can help individuals with convictions find meaningful work.

Additionally, you may want to consider seeking guidance from a legal p